In [1]:
# For a given set of parameters, generate a surface

In [2]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
import pandas as pd

from Heston_process import *

In [3]:
theta = 0.132328; kappa = 10.980797; sigma = 1.7; rho = -0.351560; v0 = 0.065690
#spot, K = 659.37, 600.0
spot, K = 1.0, 600.0/659.37
#spot, K = 659.37, 758.28
#spot, K = 659.37, 800.0
alp = 0.75
s = np.log(spot)
k = np.log(K)
x = np.log(spot)

feller_cond = Heston_feller_cond(kappa, theta, sigma)
print("Feller condition is ", feller_cond)

Feller condition is  True


In [45]:
num_grid_1d = 8
num_grid_1d = 100

alp = 0.75
C_price_bounds = [1e-3, 0.5]

k_min = np.exp(-0.2)
k_max = np.exp(0.2)
dk = (k_max - k_min)/num_grid_1d # gap between price in real world
k_list = np.log(np.linspace(k_min,k_max,num=num_grid_1d))# logstrike
logS0 = 0.0 # log-spot. Assume S0: 1
dlogS0 = np.log(1.00001)
dS0 = np.exp(dlogS0) - np.exp(logS0)

#dlogS0 = np.log(1.00001)
dt = 1/12 # monthly 
T_list = np.append(np.linspace(2.*dt,1.0,num=11), np.linspace(1.0+dt,2.0,num=12))  #
T_list = np.array([1.0])  #
#v0_list = np.linspace(np.sqrt(1e-1),np.sqrt(0.3),num=num_grid_1d) 
r = 0.0
#rho_list = np.linspace(-1.0, 1.0,num=num_grid_1d) 
#kappa_list = np.linspace(1e-3, 20.0,num=num_grid_1d) 
#theta_list = np.linspace(1e-3, 10.0,num=num_grid_1d) 
#sigma_list = np.linspace(1e-3, 20.0,num=num_grid_1d) 

pd_dict = {}
pd_dict['k'] = []; pd_dict['T'] = []; pd_dict['v0'] = []; pd_dict['rho'] = []
pd_dict['kappa'] = []; pd_dict['theta'] = []; pd_dict['sigma'] = []; pd_dict['C_price'] = []
pd_dict['delta'] = []

for k in k_list:
    for T in T_list:
        C_price = get_Heston_fft_call(k, logS0, v0, r, rho, kappa, theta, sigma, T, alp, N=2**10, dk=1e-2)
        if (C_price <= C_price_bounds[0]) | (C_price >= C_price_bounds[1]) | np.isnan(C_price):
            continue
        C_price_p = get_Heston_fft_call(k, logS0+dlogS0, v0, r, rho, kappa, theta, sigma, T, alp, N=2**10, dk=1e-2)
        pd_dict['delta'].append((C_price_p-C_price)/dS0) # delta of call 
        pd_dict['k'].append(k)
        pd_dict['T'].append(T)
        pd_dict['v0'].append(v0)
        pd_dict['rho'].append(rho)
        pd_dict['kappa'].append(kappa)
        pd_dict['theta'].append(theta)
        pd_dict['sigma'].append(sigma)
        pd_dict['C_price'].append(C_price)

df = pd.DataFrame(pd_dict)


C:\Users\golde\Documents\GitHub\projects\practices\quant\Heston\Heston_process.py:25: RuntimeWarning: overflow encountered in tanh
  coth = 1.0/np.tanh(half_gam_T)


In [46]:
df.dropna(inplace=True)
df.reset_index()
print(df.isna().sum())

k          0
T          0
v0         0
rho        0
kappa      0
theta      0
sigma      0
C_price    0
delta      0
dtype: int64


In [47]:
df.to_csv('Heston_surface', index=False)

In [48]:
k_min = np.exp(-0.2)
k_max = np.exp(0.2)
dk = (k_max - k_min)/num_grid_1d
dk

0.004026720050821881